In [1]:
    from wtypes import *
    import pytest, typing
    invalid = pytest.raises(ValidationError)

In [2]:
    def test_combo_types():
        with invalid: (Instance[range] | Integer)(1.1)
        with invalid: (Instance[range] | Integer)({})
        assert (Instance[range] | Integer)(range(10))
        assert (Instance[range] | Integer)(1)

        (Dict | Float | range)({})
        (Dict | Float | range)(1)
        (Dict | Float | range)(range(1))
        with invalid: (Dict | Float)([])

In [3]:
    def test_evented_dict():
        d = evented.Dict(); e = evented.Dict()
        assert not (d or e)

        e.link('a', d, 'b')

        e['a'] = 1

        assert d['b'] == 1
        d['b'] = 10
        assert e['a'] == 10

In [4]:
    def test_list_schema():
        assert List[typing.Union[Integer, Float]]._schema.toDict() == {'type': 'array', 'items': {'anyOf': [{'type': 'integer'}, {'type': 'number'}]}}
        assert List[AnyOf[Integer, Float]]._schema.toDict() == {'type': 'array', 'items': {'anyOf': [{'type': 'integer'}, {'type': 'number'}]}}

In [5]:
    def test_update_item():
            d = Dict[{'a': int}]({})
            assert 'a' not in d
            d['a'] = 10
            assert d['a'] == 10
            
            d = Bunch[{'a': int}]({})
            assert 'a' not in d
            d['a'] = 10
            assert d['a'] == 10

In [6]:
    def test_partial_defaults():
        class d(Dict):
            i: Integer = 20

        assert d(j=9) == {'i': 20, 'j': 9}
        assert d(i=9) == {'i': 9}


In [7]:
    def test_nested_schema():
        class c(Dict): a: object
        class d(c): b: Integer
        assert c._schema.toDict() =={'properties': {'a': {}}, 'type': 'object'}
        d._schema.toDict() =={'properties': {'a': {}, 'b': {'type': 'integer'}}, 'type': 'object'}

In [8]:
    def test_update_attr():
        
        for cls in [Bunch]:
            class Class(cls): 
                a: int = 1
            d = Class({})
            with invalid: d['a'] = 'abc'
            assert d['a'] == 1
            d.a = 10
            assert d.a == 10

In [9]:
    def test_dict():
        class Thing(Dict):
            a: Integer

        Thing._schema.toDict() == {'properties': {'a': {'type': 'integer'}}, 'required': ['a'], 'type': 'object'}
        assert Thing(a=1)
        with invalid:
            Thing(a='abc')


In [10]:
    def test_bunch():
        class Thing(Bunch):
            a: Integer

        Thing._schema.toDict() == {'type': 'object', 'properties': {'a': {'type': 'integer'}}, 'required': ['a']}
        t = Thing(a=1)
        assert t.a == 1
        with invalid:
            Thing(a='abc')


In [11]:
    def test_dict_mods():
        d = Dict[{'a': Integer, 'b': String}]({'a': 1})

        with invalid: d["a"] = 'abc'
        d['a'] = 8
        d['b'] = 'abc'
        with invalid: d['b'] = 10
        d['b'] = 'wxyz'

        assert d == {'a': 8, 'b': 'wxyz'}


In [12]:
    def test_list_mods():
        l = List[Integer, Bool]()
        with invalid: l.append(1.2)
        l.append(1)
        l.extend([1,True])
        with invalid: l.extend([1, 'abc'])
        l.extend([False, 2, 4])
        assert l == [1, 1, True, False, 2, 4]
        with invalid: l.insert(1, 'abc')
        l.insert(1, 8)
        assert l == [1, 8, 1, True, False, 2, 4]

        ml = List[Integer] + MinItems[1]

        with invalid: ml()
        with invalid: ml(['abc'])
        assert ml([1])

In [13]:
    def test_load_config():
        class BadProject(Bunch):
            tool: Dict
            __annotations__ = {'build-system': Integer}

        with invalid:
            BadProject().from_config_file('pyproject.toml')
        
        class PyProject(Bunch):
            tool: Dict
            __annotations__ = {'build-system': Dict}
        PyProject.from_config_file('pyproject.toml')

In [14]:
    def test_tuple():
        assert Tuple((1,2))
        assert Tuple([1,2])
        with invalid: Tuple[Integer, String]({})

In [15]:
    def test_complex_dict():
        with invalid: (Dict[{'a': Email}] + Required['a']+ Default[{}] + AdditionalProperties[False])({})
        with invalid: (Dict[{'a': Email}] + Default[{}] + AdditionalProperties[False])({'a': 'no'})
        assert (Dict[{'a': Email}] + Default[{}] + AdditionalProperties[False])({'a': 'no@yes'})

In [16]:
    def test_dataclass():
        class Thing(DataClass):
            a: typing.Union[Instance['range'], Integer]

        assert dataclasses.is_dataclass(Thing)
        Thing._schema.toDict() == {'type': 'object', 'properties': {'a': {'type': 'integer'}}, 'required': ['a']}
        assert Thing(a=1)
        with invalid:
            Thing(a='abc')


In [17]:
    def test_dict_additionalproperties():
        class d(Dict, additionalProperties=False):
            a: int
            b: float = 1.1
                
        with invalid: d({'c': 1})
        d({'a': 1})
        with invalid: d({'a': 1})['c'] = 1

In [18]:
    def test_evented_dc():
        class D(evented.DataClass):
            b: object = 1
        class E(evented.DataClass):
            a: object = 2
        
        d = D(); e = E()
        assert (d and e), "The defaults weren't set."

        e.link('a', d, 'b')

        e.a = 1

        assert d.b == 1
        d.b = 10
        assert e.a == 10